In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator

import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [198]:
#Importing data
data = pd.read_csv('BankChurners.csv', index_col=0)
data.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
CLIENTNUM,,,,,,,,,,,,,,,,,,,,,
768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [298]:
data.describe()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
count,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894,0.159997,0.840003
std,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691,0.365301,0.365301
min,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000008,0.000420
25%,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000,0.000099,0.999660
50%,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000,0.000181,0.999820
75%,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000,0.000337,0.999900
max,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000,0.999580,0.999990


In [87]:
#Splitting data in training and testing sets

X_train, X_test = train_test_split(data, train_size=0.9, random_state=42)
y_test = X_test[['Attrition_Flag']].replace({'Existing Customer':0,'Attrited Customer':1}).values
X_test.pop('Attrition_Flag')

y_train = X_train[['Attrition_Flag']].replace({'Existing Customer':0,'Attrited Customer':1}).values
X_train.pop('Attrition_Flag')

CLIENTNUM
710522433    Existing Customer
785500533    Existing Customer
824782008    Existing Customer
717991533    Existing Customer
718418958    Attrited Customer
                   ...        
769053033    Attrited Customer
714406158    Attrited Customer
714140133    Existing Customer
720244983    Existing Customer
827123883    Existing Customer
Name: Attrition_Flag, Length: 9114, dtype: object

In [288]:
#Classes for data preprocessing
class CatFeaturesDummies(BaseEstimator, TransformerMixin):
    'The aim of this class is to select categorical features and to create dummy variables from these features'
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        _categorical = []
        categories = {
                      'Gender':['M','F'],'Education_Level':['High School', 'Graduate', 'Uneducated', 'Unknown', 'College',
                      'Post-Graduate', 'Doctorate'], 'Marital_Status':['Married', 'Single', 'Unknown', 'Divorced'], 'Income_Category':['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
                      '$120K +', 'Unknown'], 'Card_Category':['Blue', 'Gold', 'Silver', 'Platinum']
                        
                     }
        for col in X.columns :
            if X[col].dtypes == 'O' : _categorical.append(col)
                
        Xf = pd.DataFrame()
        
        for col in categories:
            Xf = pd.concat([Xf,pd.get_dummies(pd.Categorical(X[col], categories=categories[col]), drop_first=True, prefix=col)],axis=1)
 
        return Xf
    
class NumFeaturesSelection(BaseEstimator, TransformerMixin):
    'Selection of numerical features'
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        _numerical = []
        for col in X.columns :
            if X[col].dtypes in ['int64','float64'] : _numerical.append(col)
        Xt = X.copy()[_numerical]
        return Xt

class Scaler(BaseEstimator, TransformerMixin):
    'Transform the array obtained from scaling onto a DataFrame'
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        mms = MinMaxScaler()
        return pd.DataFrame(mms.fit_transform(X), columns=X.columns)

def preprocessDataAndPredict(

                                Customer_Age, Gender, Dependent_count,
                                Education_Level, Marital_Status, Income_Category, Card_Category,
                                Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,
                                Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,
                                Avg_Open_To_Buy, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt,
                                Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio,
                                Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,
                                Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
                            ):
    
    #keep all inputs in array
    col_names = ['Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']
    test_data = [Customer_Age, Gender, Dependent_count,
                 Education_Level, Marital_Status, Income_Category, Card_Category,
                 Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,
                 Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,
                 Avg_Open_To_Buy, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt,
                 Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio,
                 Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,
                 Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2]
    print(test_data)
    
    #convert value data into numpy array
    test_data = pd.DataFrame(dict(zip(col_names, test_data)))
    
    #open file
    file = open("to_use_model.pkl","rb")
    
    #load trained model
    trained_model = joblib.load(file)
    
    #predict
    prediction = trained_model.predict(test_data)
    
    return prediction

In [289]:
#Designing pipelines
cat_pipe = Pipeline([('cfd', CatFeaturesDummies())])
num_pipe = Pipeline([('nfs',NumFeaturesSelection()), ('scaler',Scaler())])
union = FeatureUnion([('cp', cat_pipe),('np',num_pipe)])

#Models to be used
nbc = GaussianNB()
rfc = RandomForestClassifier(n_estimators = 240, criterion = 'gini', random_state = 0)
xgbc = xgb.XGBClassifier(max_depth=1,n_estimators=200,learning_rate=0.06)

In [291]:
#Model predictions
from sklearn.metrics import confusion_matrix
import pickle
import joblib

model = Pipeline([('union',union), ('model',xgbc)])
model.fit(X_train,y_train)

# save the model to disk

filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
joblib.dump(model,'to_use_model.pkl')

confusion_matrix(model.predict(X_test),y_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[868,   0],
       [  0, 145]])

In [292]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
trained_model = joblib.load('to_use_model.pkl')

In [296]:
Customer_Age, Gender, Dependent_count,Education_Level, Marital_Status, Income_Category, Card_Category,Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,Avg_Open_To_Buy, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt,Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2 = 36          ,   'M' ,  1             ,'Graduate'     ,'Married'      ,'Unknown'       ,     'Blue'   ,2             , 1                       ,                      3,                    2,       10000.0 ,                  50,              3.0,                 0.75,            2500,            35,                   2.0,                  0.45,                             0.25,0.25

#keep all inputs in array 
test_data = [Customer_Age, Gender, Dependent_count,
             Education_Level, Marital_Status, Income_Category, Card_Category,
             Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,
             Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,
             Avg_Open_To_Buy, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt,
             Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio,
             Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,
             Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2]
names = []
for elt in test_data:
    names.append([elt])

test_data = names

#convert value data into pandas dataframe
test_data = pd.DataFrame(dict(zip(col_names, test_data)))


#open file
file = open("to_use_model.pkl","rb")

#load trained model
trained_model = joblib.load(file)

#predict
prediction = trained_model.predict(test_data)

['Existing Customer', 'Attrited Customer'][prediction[0]]



'Existing Customer'